<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Cenário

Um crime foi cometido e os detetives precisam da sua ajuda! Eles te deram o relatório da cena do crime, mas de alguma forma você o perdeu. Você lembra vagamente que o crime foi um assassinato (murder) que ocorreu em algum momento no dia 15 de janeiro de 2018 e que aconteceu em SQL City. Depende de você descobrir quem é o assassino usando apenas suas habilidades em SQL e sua inteligência. Eles te fornecem uma conexão ao banco de dados do Departamento de Polícia, que tem todas as pistas que você precisará para pegar o assassino.

Use as habilidades que você desenvolveu em SQL 101 e 102, junto com qualquer recurso que desejar, para resolver o Mistério do Assassinato em SQL!

## Descobrindo tabelas no banco de dados

Começamos nossa busca para encontrar o assassino explorando o banco de dados do Departamento de Polícia. Mas você ainda não viu o banco de dados e não sabe quais são as tabelas, então, como sabe o que procurar?

Felizmente, a maioria dos sistemas de gerenciamento de bancos de dados relacionais tem essa informação armazenada em um lugar onde você pode consultá-la. Muito frequentemente, é usado um esquema especial conhecido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para armazenar informações sobre as tabelas e colunas no seu banco de dados (também conhecido como metadata). O banco de dados do Departamento de Polícia tem um information schema, com a visão `tables` que mostra quais tabelas estão disponíveis, e a visão `columns` que mostra todas as colunas de cada tabela e seus tipos de dados.

## Conecte-se ao banco de dados do Departamento de Polícia

Para começar e executar suas consultas, pressione play na célula abaixo para se conectar ao banco de dados do Departamento de Polícia.

Para executar consultas, crie uma nova célula de `Código` e escreva `%%sql` na parte superior. Depois, você pode escrever suas consultas abaixo. Veja o exemplo abaixo:
```python
%%sql

SELECT *
FROM table
```

In [ ]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

### Listando todas as tabelas no banco de dados da Polícia

#### Revise a [descrição de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primeiro, veremos todas as tabelas disponíveis para nós consultando a visão `information_schema.tables`. Vamos te dar a primeira consulta para começar, mas daqui em diante você terá que criar as consultas restantes usando seu conhecimento de SQL e sua inteligência.

Aqui está a consulta necessária para mostrar as tabelas no banco de dados do Departamento de Polícia. Copie/Cole na célula abaixo e execute para ver as tabelas disponíveis para você:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas as tabelas e suas colunas no banco de dados da Polícia

**Revise a [descrição de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

Ótimo! Agora que você sabe quais tabelas estão disponíveis, é hora de descobrir as colunas que cada tabela possui, juntamente com o tipo de dados de cada coluna. Escreva uma consulta que mostre o nome da tabela, o nome da coluna e o tipo de dados para cada tabela no banco de dados do Departamento de Polícia, usando a view `information_schema.columns`. Certifique-se de que a saída esteja ordenada por nome da tabela e, em seguida, por nome da coluna (em ordem ascendente):

*Dicas opcionais - tente usá-las apenas se você ficar **REALMENTE** preso!*

<details>
<summary>Dica 1</summary>
<p>Estruture sua consulta da mesma maneira que fizemos na consulta anterior, onde olhamos para `information_schema.tables`</p>
</details>

<details>
<summary>Dica 2</summary>
<p>Certifique-se de revisar o link para a descrição de information_schema.columns. Ele lhe dirá os nomes das colunas que você deve usar na consulta. Lembre-se, queremos o nome da tabela, o nome da coluna e o tipo de dados - verifique a descrição para encontrar os nomes apropriados!</p>
</details>

<details>
<summary>Dica 3</summary>
<p>Não se esqueça de usar a cláusula ORDER BY para ordenar os resultados da sua pesquisa. Estamos procurando ordenar ambas as colunas em ordem ascendente, o que pode ser especificado usando a palavra-chave ASC, no entanto, essa também é a ordem de classificação padrão, portanto, ASC não é necessariamente obrigatório.</p>
</details>

In [ ]:
%%sql

### Diagrama de Relacionamento de Entidades (Opcional)

Depois de fazer um pouco de trabalho de detetive e encontrar as tabelas no banco de dados do Departamento de Polícia, você descobre que há um ERD (Diagrama de Relacionamento de Entidades). Isso pode ser realmente útil em sua busca para encontrar o assassino. Se preferir, desafie-se (e suas habilidades em SQL) a continuar consultando o `information_schema` conforme necessário. Caso contrário, você pode clicar no menu suspenso abaixo para revelar o ERD que o ajudará a ver as tabelas e os relacionamentos no banco de dados do Departamento de Polícia em um relance:

<details>
  <summary>Veja o ERD do SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD para o SQL Murder Mystery"/>
</details>

In [ ]:
%%sql

___

## Resolvendo o Mistério (e verificando se você resolveu o caso)

Daqui em diante, você usará suas habilidades e as pistas no banco de dados do Departamento de Polícia para encontrar o assassino. Quando você achar que encontrou o assassino, volte a esta seção e use a célula abaixo para verificar sua resposta e ver se você encontrou o assassino:

exemplo:
```python
check_suspect("Adam Levine")
```

In [ ]:
check_suspect("Write the suspected murderer's full name here")

---

## Área de Trabalho

Use as células abaixo para escrever suas consultas e trabalhar na resolução do mistério. Quando você tiver um suspeito, verifique sua resposta usando a [célula acima desta](#resolvendo-o-mistério-e-verificando-se-você-resolveu-o-caso) e execute o nome dele através da função `check_suspect`. Se você encontrar o assassino, a função lhe dirá.

Não se esqueça de usar `%%sql` na parte superior das células de `Código` que você criar para que suas consultas SQL funcionem. Boa sorte, detetives!

In [ ]:
%%sql
